# Combine all data from different versions

Use this notebook to combine all the previous data coming from different batches.

In [ ]:
import os
import pandas as pd

In [ ]:
# Load the data from the xls file
base_dir = "../EoIs/Inputs/"
versions = ["Version1_Batches1-6", "Version2_Batches7-9", "Version3_Batches10-11"]

In [ ]:
# change from xls to csv
for version in versions:
    for file_name in os.listdir(base_dir + version):
        # if filename starts with ~ then it is a temporary file created by excel, skip it
        if file_name.startswith("~"):
            continue
        # then concat the data from the file to the data
        if file_name.endswith(".xlsx"):
            data = pd.read_excel(base_dir + version + "/" + file_name)
            data.to_csv(base_dir + version + "/" + file_name[:-5] + ".csv", index=False)

In [ ]:
# concatenate the data in all csv files in a directory
data_v1 = pd.DataFrame()
data_v2 = pd.DataFrame()
data_v3 = pd.DataFrame()

for version in versions:
    data = pd.DataFrame()
    for file_name in os.listdir(base_dir + version):
        # if filename starts with ~ then it is a temporary file created by excel, skip it
        if file_name.startswith("~"):
            continue
        # then concat the data from the file to the data
        if file_name.endswith(".csv"):
            data = pd.concat([data, pd.read_csv(base_dir + version + "/" + file_name)], ignore_index=True)
    if version == versions[0]:
        data_v1 = data
    elif version == versions[1]:
        data_v2 = data
    elif version == versions[2]:
        data_v3 = data

In [ ]:
# Check the data shape
print(data_v1.shape)
print(data_v2.shape)
print(data_v3.shape)

In [ ]:
# Check the data columns
data_v1.columns

In [ ]:
data_v2.columns

In [ ]:
data_v3.columns

In [ ]:
# Optional: Remove columns with sensitive information
sensitive_columns = ["First name", "Last name", "Email address", "IP Address", "User Agent", "URL"]
data_v1 = data_v1.drop(columns=sensitive_columns)
# First version of the data does not have the following columns
sensitive_columns = ['Companies House Registration no:', 'Organisation Size', 'Organisation Type', 'Address', 'Postcode', 'Country', "First name", "Last name", "Email address", "IP Address", "User Agent", "URL"]
data_v2 = data_v2.drop(columns=sensitive_columns)
data_v3 = data_v3.drop(columns=sensitive_columns)

In [ ]:
# Check the data columns and shape after removing sensitive information
data_v1.head(2)

In [ ]:
# change data_v1 column names to match it with the latest versions
data_v1 = data_v1.rename(columns={
    "Position": "Job Title",
    "Agriculture": "Agriculture and food processing",
    "Transportation": "Transportation, including logistics and warehousing",
    "Process deficiencies and storage": "Technical infrastructure challenges"
    })

In [ ]:
data_v1.drop(columns=[
    'Please add any other information that you would like to provide as part of this Expression of Interest form? (max 200 words)',
    'Please tick to confirm Innovate UK Business Connect have your permission to process your data. You can view Innovate UK Business Connect\'s Privacy Policy here.',
    'Please tick to confirm The Alan Turing Institute have your permission to process your data. You can view The Alan Turing Institute\'s Privacy Policy here.',
    'Please tick to confirm Innovate UK KTN have your permission to process your data. You can view Innovate UK KTN\'s Privacy Policy here.'
], inplace=True)

In [ ]:
data_v2.drop(columns = [
    'Please tick to confirm that you understand as this is a BridgeAI project, the Alan Turing Institute (Privacy Policy) will be sharing your personal data, which is collected on behalf of the BridgeAI programme, with Innovate UK (Privacy Notice), Innovate UK Business Connect (Privacy Policy), BSI Group (Privacy Notice), Hartree Centre (Privacy Notice), and Digital Catapult (Privacy Notice) for the purpose described under the Innovate UK Business Connect privacy notice.',
    'Please tick to confirm Innovate UK Business Connect have your permission to process your data. You can view Innovate UK Business Connect\'s Privacy Policy here.'
], inplace=True)

In [ ]:
data_v3.drop(columns=[
    'Please tick to confirm that you understand as this is a BridgeAI project, the Alan Turing Institute (Privacy Policy) will be sharing your personal data, which is collected on behalf of the BridgeAI programme, with Innovate UK (Privacy Notice), Innovate UK Business Connect (Privacy Policy), BSI Group (Privacy Notice), Hartree Centre (Privacy Notice), and Digital Catapult (Privacy Notice) for the purpose described under the Innovate UK Business Connect privacy notice.',
    'Please tick to confirm Innovate UK Business Connect have your permission to process your data. You can view Innovate UK Business Connect\'s Privacy Policy here.',
], inplace=True)

In [ ]:
merged_data = pd.concat([data_v1, data_v2, data_v3], axis=0, join='outer', ignore_index=True)

In [ ]:
merged_data.columns

In [ ]:
merged_data.shape

In [ ]:
merged_data.fillna("N/A", inplace=True)

In [ ]:
merged_data.to_csv("../EoIs/merged_data.csv", index=False)

In [ ]:
# if the "Organisation Name" column has a value more than once, drop duplicates
unique_merged_data = merged_data.drop_duplicates(subset="Organisation Name", keep="first")

In [ ]:
# also save this data
unique_merged_data.to_csv("../EoIs/unique_merged_data.csv", index=False)